# Tuning Script

In [1]:
# Parameters subject to change in every script - involved in naming

MODEL = 'tfagnn'
TUNER = 'jiaochengb'

---

In [2]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

Mounted at /content/drive


In [3]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    home_directory = './drive/MyDrive/LAB/AFLBrownlowPredictor2024/' # my home directory is stored in ./LAB of google drive
    if MODEL == 'ebr':
        !pip install interpret==0.5.0
    elif MODEL == 'cbr':
        !pip install catboost
    elif MODEL in ['tf', 'mlp', 'mlpagnn', 'tfagnn']:
        !pip install PyTorch2Sklearn==0.2.3
    !pip install JXAutoML==0.4.3
else:
    home_directory = '../../'

from JXAutoML.JiaoChengB import JiaoChengB as Tuner

import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.5 MB/s eta 0:00:00


In [4]:
# Parameters that are same for every script
SEED = 42

TASK_TYPE = 'Regression'  # this task is a classification task

# we have decided to tune features as hp
TUNE_FEATURES_AS_HYPERPARAMETERS = False

PyTorchModel = True

PyTorchGraphModel = True

---

In [5]:
# model to use to tune
from PyTorch2Sklearn.Transformer_AGNN import Transformer_AGNN as clf
from PyTorch2Sklearn.environment import *
from PyTorch2Sklearn.utils.data import GraphDataFactory


# what values to try for each hyperparameter
parameter_choices = {
    'hidden_dim': (16, 32, 64, 128, 256),
    'num_transformer_layers': (1, 2, 3),
    'num_graph_layers': (1, 2),
    'num_mlp_layers': (1, 2, 3),
    'graph_nhead': (2, 4, 8, 16),
    'dropout': (0, 0.05, 0.1, 0.2),
    'nhead': (2, 4, 8, 16),
    'share_embedding_mlp': (False, True),
    'use_cls': (False, True),
    'epochs': (5, 10, 20, 30),
    'lr': (1e-5, 1e-4, 1e-3, 1e-2, 1e-1),
    'batchnorm': (False, True),
    'grad_clip': (False, True),
}

# what values to set non-tuneable parameters/hyperparameters
non_tunable_hyperparameters_dict = {
    'random_state': SEED,
    'loss': nn.MSELoss(),
    'rootpath': './',
    'GraphDataFactory': GraphDataFactory,
    'output_dim': 1,
    'mode': 'Regression',
    'verbose': True,
    'input_dim': 37
}

# what order to tune hyperparameters in (JiaoCheng specific)
tuning_order = [
    # 'features',
    'num_transformer_layers',
    'graph_nhead',
    'num_graph_layers',
    'num_mlp_layers',
    'nhead',
    'hidden_dim',
    'dropout',
    'share_embedding_mlp',
    'use_cls',
    'batchnorm',
    'grad_clip',
    'lr',
    'epochs',
]

# what order to tune hyperparameters in (JiaoCheng specific)
default_hyperparameter_values = {
    # 'features': 36,
    'num_transformer_layers': 1,
    'num_mlp_layers': 1,
    'num_graph_layers': 1,
    'graph_nhead': 4,
    'nhead': 4,
    'hidden_dim': 16,
    'dropout': 0.1,
    'share_embedding_mlp': True,
    'use_cls': False,
    'batchnorm': False,
    'grad_clip': False,
    'lr': 1e-2,
    'epochs': 10,
}

---

Read in and Prepare Data

In [6]:
train_data = pd.read_csv(
    f'{home_directory}data/curated/modelling/graph_train.csv')
val_data = pd.read_csv(f'{home_directory}data/curated/modelling/graph_val.csv')
test_data = pd.read_csv(
    f'{home_directory}data/curated/modelling/graph_test.csv')

train_X = train_data.drop(
    columns=['target', 'rank', 'Unnamed: 0', 'Player', 'Brownlow Votes', 'Supercoach', 'HomeAway'])
train_y = train_data[['target', 'idx']]
val_X = val_data.drop(
    columns=['target', 'rank', 'Unnamed: 0', 'Player', 'Brownlow Votes', 'Supercoach', 'HomeAway'])
val_y = val_data[['target', 'idx']]
test_X = test_data.drop(
    columns=['target', 'rank', 'Unnamed: 0', 'Player', 'Brownlow Votes', 'Supercoach', 'HomeAway'])
test_y = test_data[['target', 'idx']]

In [7]:
# Import feature importance ordering

with open(f'{home_directory}models/feature_importance_ordering.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

Set parameters

In [8]:
# initialisation
tuner = Tuner()

print('---')

# define what model we are tuning
tuner.read_in_model(
    clf, TASK_TYPE, pytorch_model=PyTorchModel, optimised_metric='r2', pytorch_graph_model=PyTorchGraphModel)

print('---')

# read in the data for training and validation
tuner.read_in_data(train_X, train_y, val_X, val_y, test_X, test_y)

print('---')

# set what hp values to tune
tuner.set_hyperparameters(parameter_choices)
# WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up hp values that need to be changed from default but NOT to be tuned
tuner.set_non_tuneable_hyperparameters(non_tunable_hyperparameters_dict)

print('---')

# set up feature importance ordering

if TUNE_FEATURES_AS_HYPERPARAMETERS:
    tuner.set_features(feature_importance_ordering)
    # WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up the order of hyperparameters when iteratively tuning using JiaoCheng
tuner.set_tuning_order(tuning_order)

print('---')

# set up the default hp values for first iteration of tuning JiaoCheng
tuner.set_hyperparameter_default_values(default_hyperparameter_values)

print('---')

try:  # try to read in previous results to continue tuning ...
    tuner.read_in_tuning_result_df(
        f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')
except:
    print('No previous tuning data read in')

print('---')

# set up where to save the tuning result csv
tuner.set_tuning_result_saving_address(
    f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')

print('---')

# set up where to save the current best model
tuner.set_best_model_saving_address(
    f'{home_directory}models/tmp_models/{TUNER}_{MODEL}')

JiaoCheng Initialised
---
Successfully read in model <class 'PyTorch2Sklearn.Transformer_AGNN.Transformer_AGNN'>, which is a Regression model optimising for r2
---
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
---
Successfully recorded hyperparameter choices
---
Successfully recorded non_tuneable_hyperparameter choices
---
---
---
---
Successfully read in tuning result of 53 rows, for 53.0 combos
---
Successfully set tuning output address
---
Successfully set best model output address


In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
# begin tuning ... (or continue training)

tuner.tune()


Default combo: [0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 3, 0, 0] 


FEATURE_INDEX: 0 (num_transformer_layers) ROUND 1

Round 1 
Hyperparameter: num_transformer_layers (index: 1) 

As new Best Combo (0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 3, 0, 0) was read in, best_clf is set to None
Already Trained and Tested combination {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 4, 'dropout': 0.1, 'nhead': 4, 'share_embedding_mlp': True, 'use_cls': False, 'epochs': 10, 'lr': 0.01, 'batchnorm': False, 'grad_clip': False}, which had val score of 0.8973
        Current best combo: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 4, 'dropout': 0.1, 'nhead': 4, 'share_embedding_mlp': True, 'use_cls': False, 'epochs': 10, 'lr': 0.01, 'batchnorm': False, 'grad_clip': False} with val score 0.8973. 
        Has trained 53 of 1843200 combinations so far
Already Trained and Tested combination {'hidden_dim': 16

100%|██████████| 20/20 [09:18<00:00, 27.94s/it]


Trained and Tested combination 54 of 1843200: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.0001, 'batchnorm': False, 'grad_clip': False}, taking 559.7 seconds to get val score of 0.9091
        Current best combo: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.001, 'batchnorm': False, 'grad_clip': False} with val score 0.9157
Already Trained and Tested combination {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.001, 'batchnorm': False, 'grad_clip': False}, which had val score of 0.9157
        Current best combo: {'h

100%|██████████| 20/20 [09:20<00:00, 28.04s/it]


Trained and Tested combination 55 of 1843200: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.01, 'batchnorm': False, 'grad_clip': False}, taking 561.93 seconds to get val score of 0.9106
        Current best combo: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.001, 'batchnorm': False, 'grad_clip': False} with val score 0.9157


...

Trained and Tested combination 78 of 1843200: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 4, 'dropout': 0.1, 'nhead': 4, 'share_embedding_mlp': True, 'use_cls': False, 'epochs': 5, 'lr': 0.01, 'batchnorm': False, 'grad_clip': False}, taking 132.82 seconds to get val score of 0.8985
        Current best combo: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.001, 'batchnorm': False, 'grad_clip': False} with val score 0.9157
Already Trained and Tested combination {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 4, 'dropout': 0.1, 'nhead': 4, 'share_embedding_mlp': True, 'use_cls': False, 'epochs': 10, 'lr': 0.01, 'batchnorm': False, 'grad_clip': False}, which had val score of 0.8973
        Current best combo: {'hi

100%|██████████| 20/20 [08:42<00:00, 26.10s/it]


Trained and Tested combination 79 of 1843200: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 4, 'dropout': 0.1, 'nhead': 4, 'share_embedding_mlp': True, 'use_cls': False, 'epochs': 20, 'lr': 0.01, 'batchnorm': False, 'grad_clip': False}, taking 522.94 seconds to get val score of 0.7588
        Current best combo: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.001, 'batchnorm': False, 'grad_clip': False} with val score 0.9157


100%|██████████| 30/30 [12:53<00:00, 25.77s/it]


Trained and Tested combination 80 of 1843200: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 4, 'dropout': 0.1, 'nhead': 4, 'share_embedding_mlp': True, 'use_cls': False, 'epochs': 30, 'lr': 0.01, 'batchnorm': False, 'grad_clip': False}, taking 773.96 seconds to get val score of 0.7268
        Current best combo: {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4, 'share_embedding_mlp': False, 'use_cls': False, 'epochs': 20, 'lr': 0.001, 'batchnorm': False, 'grad_clip': False} with val score 0.9157

Best combo after this hyperparameter: (0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 0, 0) , UPDATED SINCE LAST HYPERPARAMETER


Round 1 
Hyperparameter: num_transformer_layers (index: 1) 

Already Trained and Tested combination {'hidden_dim': 16, 'num_transformer_layers': 1, 'num_graph_layers': 1, 'num_mlp_layers': 1, 'graph_nhead': 8, 'dropout': 0, 'nhead': 4,